# Actor-Critic


<img src="ac_scheme.png">

«Критик» оценивает функцию полезности. Это может быть полезность действия ($Q$) или полезность состояния ($V$).

«Актор» обновляет распределение стратегии в направлении, предложенном Критиком.


Псевдокод для $Q$ версии алгоритма приведен ниже:
<img src="alg_ac.png" style="width: 700px">


### Задание 1. Заполните пропуски для $V$ версии алгоритма Actor-Critic

In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

learning_rate = 0.0001
gamma = 0.98

In [2]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.data = []

        # создаем сеть с двумя головами для pi и V (подсказка - смотрим на методы pi и v)
        ########## Solution ############
        self.fc1 = nn.Linear(4, 256)
        self.fc_pi = nn.Linear(256, 2)
        self.fc_v = nn.Linear(256, 1)
        ################################

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self, x, softmax_dim=0):
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim)
        return prob

    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v

    def put_data(self, transition):
        self.data.append(transition)

    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, done_lst = [], [], [], [], []

        for transition in self.data:
            # собираем батч из сохранненых кортежей
            # не забывайте сделать клиппинг вознаграждений!
            ########## Solution ############
            s, a, r, s_prime, done = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r / 100])
            s_prime_lst.append(s_prime)
            ################################
            # не забываем про окончание эпизода
            done_mask = 0.0 if done else 1.0
            done_lst.append([done_mask])

        s_batch, a_batch, r_batch, s_prime_batch, done_batch = torch.tensor(s_lst, dtype=torch.float), torch.tensor(
            a_lst), \
                                                               torch.tensor(r_lst, dtype=torch.float), torch.tensor(
            s_prime_lst, dtype=torch.float), \
                                                               torch.tensor(done_lst, dtype=torch.float)
        self.data = []
        return s_batch, a_batch, r_batch, s_prime_batch, done_batch

    def train_net(self):
        # получаем батч
        # s, a, r, s_prime, done =
        ########## Solution ############
        s, a, r, s_prime, done = self.make_batch()
        ################################

        # считаем td target(r + gamma ...) и td ошибку (td_target - v)
        # td_target = 
        # delta = 
        ########## Solution ############
        td_target = r + gamma * self.v(s_prime) * done
        delta = td_target - self.v(s)
        ################################

        # получаем распределение
        pi = self.pi(s, softmax_dim=1)
        pi_a = pi.gather(1, a)

        # считаем td_loss (подсказка можно воспользоваться smooth_l1_loss) (не забыаем про лишние градиенты!)
        # td_loss = 
        td_loss = F.smooth_l1_loss(self.v(s), td_target.detach())
        # считаем loss актора (-pi_a) * delta (не забыаем про лишние градиенты!)
        # a_loss =  
        a_loss = -torch.log(pi_a) * delta.detach()

        # вызываем backward и делаем шаг оптимизатора для каждого лосса
        ########## Solution ############
        self.optimizer.zero_grad()
        loss = a_loss + td_loss
        loss.mean().backward()
        self.optimizer.step()
        ################################

### Задание 2. Подбираем гиперпараметры и строим графики. Как влияет клиппинг вознаграждений на итоговую сходимость?

In [4]:
def run():
    env = gym.make('CartPole-v1')
    model = ActorCritic()
    n_rollout = 5
    print_interval = 20
    score = 0.0

    for n_epi in range(1001):
        done = False
        s = env.reset()
        while not done:
            for t in range(n_rollout):
                prob = model.pi(torch.from_numpy(s).float())
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                model.put_data((s, a, r, s_prime, done))

                s = s_prime
                score += r

                if done:
                    break

            model.train_net()

        if n_epi % print_interval == 0 and n_epi != 0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score / print_interval))
            score = 0.0
    env.close()


run()

# of episode :20, avg score : 22.2
# of episode :40, avg score : 21.7
# of episode :60, avg score : 22.4
# of episode :80, avg score : 27.1
# of episode :100, avg score : 36.6
# of episode :120, avg score : 44.2
# of episode :140, avg score : 47.5
# of episode :160, avg score : 52.1
# of episode :180, avg score : 60.0
# of episode :200, avg score : 90.0
# of episode :220, avg score : 114.8
# of episode :240, avg score : 105.7
# of episode :260, avg score : 124.5
# of episode :280, avg score : 207.2
# of episode :300, avg score : 217.2
# of episode :320, avg score : 188.5
# of episode :340, avg score : 248.6
# of episode :360, avg score : 238.3
# of episode :380, avg score : 217.2
# of episode :400, avg score : 145.6
# of episode :420, avg score : 124.8
# of episode :440, avg score : 143.0
# of episode :460, avg score : 196.6
# of episode :480, avg score : 233.0
# of episode :500, avg score : 212.7
# of episode :520, avg score : 253.3
# of episode :540, avg score : 225.3
# of episode :5